In [1]:
import pickle
import os
import pandas as pd
import numpy as np
import sqlite3
from tqdm import tqdm
import copy
import time

from datetime import datetime, timedelta

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

def normalize3(a, min_a=None, max_a=None):
    if min_a is None: min_a, max_a = np.min(a, axis=0), np.max(a, axis=0)
    return ((a - min_a) / (max_a - min_a + 0.0001)), min_a, max_a

def denormalize3(a_norm, min_a, max_a):
    return a_norm * (max_a - min_a + 0.0001) + min_a

def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

def init_db(feature_set, db_name="masters_data.db", table_name="severity_trending"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Create table if it does not exist
    columns = ", ".join([feature_name.replace(" ", "_") for feature_name in feature_set])
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            {columns}
        )
    """)

    conn.commit()
    conn.close()

def init_db_timeconst(feature_set, db_name="masters_data.db", table_name="severity_trending"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Create table if it does not exist
    columns = ", ".join([feature_name.replace(" ", "_") for feature_name in feature_set])
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            {columns}
        )
    """)

    conn.commit()
    conn.close()

def trend_savedb(data, feature_set, db_name="data.db", table_name="sensor_data"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    timestamp = datetime.now().replace(second=0, microsecond=0).isoformat()
    cursor.execute(f"""
        INSERT INTO {table_name} (timestamp, {', '.join([feature_name.replace(" ", "_") for feature_name in feature_set])})
        VALUES (?, {', '.join(['?' for _ in range(len(feature_set))])})
    """, (timestamp, *data))
    
    conn.commit()
    conn.close()

def timeseries_savedb(df_timestamp, data, feature_set, db_name="data.db", table_name="sensor_data"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    # Generate timestamp
    timestamp = df_timestamp.isoformat()
    
    # Build column names for features, replacing spaces with underscores
    feature_columns = ', '.join([feature_name.replace(" ", "_") for feature_name in feature_set])
    placeholders = ', '.join(['?' for _ in range(len(feature_set))])
    
    # Upsert using INSERT OR REPLACE
    # Note: Your table must have a UNIQUE constraint on the timestamp column.
    sql = f"""
        INSERT OR REPLACE INTO {table_name} (timestamp, {feature_columns})
        VALUES (?, {placeholders})
    """
    cursor.execute(sql, (timestamp, *data))
    
    conn.commit()
    conn.close()

def batch_timeseries_savedb(df_timestamps, data, feature_set, db_name="data.db", table_name="sensor_data"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Convert timestamps to ISO format
    timestamps = [pd.to_datetime(ts).isoformat() for ts in df_timestamps]
    
    # Build column names for features, replacing spaces with underscores
    feature_columns = ', '.join([feature_name.replace(" ", "_") for feature_name in feature_set])
    placeholders = ', '.join(['?' for _ in range(len(feature_set)+1)])  # 30 features + 1 timestamp
    
    # Prepare batch data
    batch_data = [(timestamps[i], *data[i]) for i in range(data.shape[0])]
    
    # Upsert using INSERT OR REPLACE (Ensure UNIQUE constraint on timestamp in your DB schema)
    sql = f"""
        INSERT OR REPLACE INTO {table_name} (timestamp, {feature_columns})
        VALUES ({placeholders})
    """
    
    cursor.executemany(sql, batch_data)
    conn.commit()
    conn.close()

def fetch_between_dates(start_date, end_date, db_name="data.db", table_name="sensor_data"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    cursor.execute(f"""
        SELECT * FROM {table_name} WHERE timestamp BETWEEN ? AND ?
    """, (start_date, end_date))
    
    rows = cursor.fetchall()
    conn.close()

    if not rows:
        return np.array([])
    
    return np.array(rows)

def convert_timestamp(timestamp_str):
    dt = datetime.fromisoformat(timestamp_str)
    return pd.Timestamp(dt.strftime('%Y-%m-%d %H:%M:%S'))

In [2]:
feature_set = ['Active Power', 'Reactive Power', 'Governor speed actual', 'UGB X displacement', 'UGB Y displacement',
    'LGB X displacement', 'LGB Y displacement', 'TGB X displacement',
    'TGB Y displacement', 'Stator winding temperature 13',
    'Stator winding temperature 14', 'Stator winding temperature 15',
    'Surface Air Cooler Air Outlet Temperature',
    'Surface Air Cooler Water Inlet Temperature',
    'Surface Air Cooler Water Outlet Temperature',
    'Stator core temperature', 'UGB metal temperature',
    'LGB metal temperature 1', 'LGB metal temperature 2',
    'LGB oil temperature', 'Penstock Flow', 'Turbine flow',
    'UGB cooling water flow', 'LGB cooling water flow',
    'Generator cooling water flow', 'Governor Penstock Pressure',
    'Penstock pressure', 'Opening Wicked Gate', 'UGB Oil Contaminant',
    'Gen Thrust Bearing Oil Contaminant']

with open('normalize_2023.pickle', 'rb') as handle:
    normalize_obj = pickle.load(handle)
    min_a, max_a = normalize_obj['min_a'], normalize_obj['max_a']
min_a = min_a.astype(float)
max_a = max_a.astype(float)

model_array = ["Attention", "DTAAD", "MAD_GAN", "TranAD", "DAGMM", "USAD", "OmniAnomaly"]
with open('model_thr.pickle', 'rb') as handle:
    model_thr = pickle.load(handle)

measured_horizon = 60 * 2 * 1

init_db_timeconst(feature_set, "db/original_data.db", "original_data")
init_db_timeconst(feature_set, "db/severity_trendings.db", "severity_trendings")
init_db_timeconst(feature_set, "db/severity_trendings.db", "original_sensor")
for model_name in model_array:
    init_db_timeconst(feature_set, "db/pred_data.db", model_name)
    init_db_timeconst(feature_set, "db/threshold_data.db", model_name)

In [3]:
num_sensors = 30
num_timesteps = 20

In [8]:
count = 0
conn = sqlite3.connect("db/original_data.db")
cursor = conn.cursor()
cursor.execute(f"""SELECT * FROM original_data order by rowid desc LIMIT 1""")
rows = cursor.fetchall()
conn.close()
df_timestamp_last = np.datetime64(np.array(rows)[:, 1][0])  #np.datetime64('now') - np.timedelta64(2, 'h') #np.datetime64(np.array(rows)[:, 1][0]) 
while True:
    print("Executing task... " + str(count))

    df_feature = np.zeros((num_timesteps, num_sensors))
    temp_ypreds = []
    for idx, _ in enumerate(model_array):
        temp_ypreds.append(np.zeros((num_timesteps, num_sensors)))

    df_feature[0] = np.random.uniform(min_a, max_a)
    for idx, _ in enumerate(model_array):
        temp_ypreds[idx][0] = df_feature[0].copy()

    for t in range(1, num_timesteps):
        prev_values = df_feature[t - 1]
        perturbation = np.random.uniform(-0.05, 0.05, size=(num_sensors)) * prev_values  # Max 5% change
        new_values = prev_values + perturbation
        new_values = np.clip(new_values, min_a, max_a)  # Ensure within bounds
        df_feature[t] = new_values
        for idx, _ in enumerate(model_array):
            temp_ypreds[idx][t] = (np.random.uniform(-0.1, 0.1, size=(num_sensors)) * new_values) + new_values

    df_feature_mean = trunc(np.mean(df_feature, axis=0), decs=2)
    trend_data = np.random.poisson(5, num_sensors)

    threshold_percentages = {}
    for idx, _ in enumerate(model_array):
        temp_thre = {}
        for feat_name in feature_set:
            temp_thre[feat_name] = int(np.random.uniform(0, 7))
        threshold_percentages[idx] = temp_thre

    now_time = np.datetime64('now')
    past_time = now_time - np.timedelta64(2, 'h')
    df_timestamp = np.arange(past_time, now_time, np.timedelta64(6, 'm'))

    mask = df_timestamp > df_timestamp_last
    df_feature = df_feature[mask]
    df_timestamp = df_timestamp[mask]
    for i in range(len(model_array)):
        temp_ypreds[i] = temp_ypreds[i][mask]

    batch_timeseries_savedb(df_timestamp, trunc(df_feature, decs=2), feature_set, "db/original_data.db", "original_data")
    for idx_model, (model_name) in enumerate(model_array):
        batch_timeseries_savedb(df_timestamp, trunc(temp_ypreds[idx_model], decs=2), feature_set, "db/pred_data.db", model_name) 

    df_timestampi = pd.to_datetime(df_timestamp[-1])
    for model_idx, model_name in enumerate(model_array):
        timeseries_savedb(df_timestampi, trunc(np.array(list(threshold_percentages[model_idx].values())), decs=2), feature_set, "db/threshold_data.db", model_name) 

    timeseries_savedb(df_timestampi, trend_data.tolist(), feature_set, "db/severity_trendings.db", "severity_trendings") 
    timeseries_savedb(df_timestampi, df_feature_mean, feature_set, "db/severity_trendings.db", "original_sensor") 

    # DONT REMOVE THIS
    df_timestamp_last = df_timestamp[-1]
    count = count + 1
    time.sleep(180)

Executing task... 0
Executing task... 1
Executing task... 2
Executing task... 3
Executing task... 4
Executing task... 5
Executing task... 6
Executing task... 7
Executing task... 8
Executing task... 9
Executing task... 10
Executing task... 11
Executing task... 12
Executing task... 13
Executing task... 14
Executing task... 15
Executing task... 16
Executing task... 17
Executing task... 18
Executing task... 19
Executing task... 20
Executing task... 21
Executing task... 22
Executing task... 23
Executing task... 24
Executing task... 25
Executing task... 26
Executing task... 27
Executing task... 28
Executing task... 29
Executing task... 30
Executing task... 31
Executing task... 32
Executing task... 33
Executing task... 34
Executing task... 35
Executing task... 36
Executing task... 37
Executing task... 38
Executing task... 39
Executing task... 40
Executing task... 41
Executing task... 42
Executing task... 43
Executing task... 44
Executing task... 45
Executing task... 46
Executing task... 47
Ex

KeyboardInterrupt: 